In [71]:
import os
import cv2
import torch
import numpy as np
from torchvision.models import resnet50
import torchvision.transforms as transforms
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report
from sklearn.model_selection import train_test_split
import torchvision.transforms as transforms
from skimage.feature import local_binary_pattern


In [72]:
# Define the paths to your flooded and non-flooded image folders
flooded_image_folder = 'dataset/flooded/'
non_flooded_image_folder = 'dataset/non-flooded/'

In [73]:
# Function to read and preprocess the images
def preprocess_image(image_path):
    # Read the image
    image = cv2.imread(image_path)
    
    # Apply preprocessing steps
    # For example, you can resize the image to a specific size (e.g., 256x256)
    resized_image = cv2.resize(image, (256, 256))
    
    # You can apply additional preprocessing steps here, such as noise removal, image enhancement, etc.
    
    # Normalize the pixel values to a specific range (e.g., 0-1)
    normalized_image = resized_image.astype(np.float32) / 255.0
    
    return normalized_image

In [74]:
# Read and preprocess the flooded images
flooded_images = []
for image_file in os.listdir(flooded_image_folder):
    image_path = os.path.join(flooded_image_folder, image_file)
    preprocessed_image = preprocess_image(image_path)
    flooded_images.append(preprocessed_image)

In [75]:
# Read and preprocess the non-flooded images
non_flooded_images = []
for image_file in os.listdir(non_flooded_image_folder):
    image_path = os.path.join(non_flooded_image_folder, image_file)
    preprocessed_image = preprocess_image(image_path)
    non_flooded_images.append(preprocessed_image)

In [76]:
# Convert the image lists to NumPy arrays
flooded_images = np.array(flooded_images)
non_flooded_images = np.array(non_flooded_images)

In [77]:
# Print the shapes of the preprocessed image arrays
print("Shape of flooded images array:", flooded_images.shape)
print("Shape of non-flooded images array:", non_flooded_images.shape)

Shape of flooded images array: (425, 256, 256, 3)
Shape of non-flooded images array: (425, 256, 256, 3)


In [78]:
def extract_cnn_features(image):
    # Preprocess the image for ResNet-50
    transform = transforms.Compose([
        transforms.ToPILImage(),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
    ])
    
    # Convert the image to uint8 data type
    image = (image * 255).astype(np.uint8)
    
    # Apply the transformations
    preprocessed_image = transform(image)
    
    # Load the pre-trained ResNet-50 model
    model = resnet50(pretrained=True)
    
    # Remove the last fully connected layer from the model
    features_model = torch.nn.Sequential(*list(model.children())[:-1])
    
    # Set the model to evaluation mode
    features_model.eval()
    
    # Extract features from the preprocessed image
    with torch.no_grad():
        features = features_model(preprocessed_image.unsqueeze(0))
    
    # Flatten the features
    flattened_features = features.view(features.size(0), -1).numpy()
    
    return flattened_features

In [79]:
# Function to extract features using Local Binary Patterns (LBP)
def extract_lbp_features(image):
    # Convert the image to grayscale
    grayscale_image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    
    # Compute LBP features
    lbp = local_binary_pattern(grayscale_image, 8, 1, method='uniform')
    histogram, _ = np.histogram(lbp.ravel(), bins=np.arange(0, 10), range=(0, 10))
    normalized_histogram = histogram.astype(np.float32) / np.sum(histogram)
    
    return normalized_histogram

In [80]:
# Apply feature extraction methods to the flooded images
flooded_cnn_features = []
flooded_lbp_features = []

for image in flooded_images:
    flooded_cnn_features.append(extract_cnn_features(image))
    flooded_lbp_features.append(extract_lbp_features(image))



# Convert the feature lists to NumPy arrays
flooded_cnn_features = np.array(flooded_cnn_features)
flooded_lbp_features = np.array(flooded_lbp_features)

# Print the shapes of the feature arrays
print("Shape of flooded CNN features array:", flooded_cnn_features.shape)
print("Shape of flooded CNN features array:", flooded_lbp_features.shape)

c:\ProgramData\Anaconda3\lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
c:\ProgramData\Anaconda3\lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet50_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet50_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


Shape of flooded CNN features array: (425, 1, 2048)
Shape of flooded CNN features array: (425, 9)


In [81]:
# Apply feature extraction methods to the non-flooded images
non_flooded_cnn_features = []
non_flooded_lbp_features = []


for image in non_flooded_images:
    non_flooded_cnn_features.append(extract_cnn_features(image))
    non_flooded_lbp_features.append(extract_lbp_features(image))

# Convert the feature lists to NumPy arrays
non_flooded_cnn_features = np.array(non_flooded_cnn_features)
non_flooded_lbp_features = np.array(non_flooded_lbp_features)

# Print the shapes of the feature arrays
print("Shape of non-flooded CNN features array:", non_flooded_cnn_features.shape)
print("Shape of non-flooded CNN features array:", non_flooded_lbp_features.shape)

Shape of non-flooded CNN features array: (425, 1, 2048)
Shape of non-flooded CNN features array: (425, 9)


In [93]:
def random_forest_classification (cnn_flooded_features, cnn_non_flooded_features, lbp_flooded_features, lbp_non_flooded_features):
    
    
    # Combine the CNN features
    cnn_features = np.concatenate((cnn_flooded_features, cnn_non_flooded_features), axis=0)
    cnn_features = cnn_features.reshape(cnn_features.shape[0], -1)

    # Combine the LBP features
    lbp_features = np.concatenate((lbp_flooded_features, lbp_non_flooded_features), axis=0)

    # Combine the features
    features = np.concatenate((cnn_features, lbp_features), axis=1)

    # Combine the labels for training
    labels = np.concatenate((np.ones(len(cnn_flooded_features)), np.zeros(len(cnn_non_flooded_features))), axis=0)


    # Split the data into training and testing sets
    X_train, X_test, y_train, y_test = train_test_split(features, labels, test_size=0.1, random_state=42)
    print(X_test)

    # Initialize and train the Random Forest classifier
    rf_classifier = RandomForestClassifier(n_estimators=100, random_state=42)
    rf_classifier.fit(X_train, y_train)

    # Make predictions on the test set
    y_pred = rf_classifier.predict(X_test)

    # Evaluate the performance of the classifier
    classification_metrics = classification_report(y_test, y_pred)
    print(classification_metrics)
    return rf_classifier


In [96]:
rf_classifier = random_forest_classification(flooded_cnn_features,non_flooded_cnn_features,flooded_lbp_features,non_flooded_lbp_features)

[[0.17089361 0.34173584 0.52135956 ... 0.05836487 0.10435486 0.35571289]
 [0.13594151 0.03875391 0.59357691 ... 0.07199097 0.09033203 0.245224  ]
 [0.51508635 0.59531248 0.63556683 ... 0.06089783 0.10874939 0.36103821]
 ...
 [0.40926659 0.22301556 0.52010506 ... 0.05654907 0.07208252 0.43473816]
 [0.62423277 0.47361854 0.60128808 ... 0.05465698 0.10926819 0.37112427]
 [0.55211419 0.60877651 0.15683565 ... 0.07328796 0.0738678  0.19538879]]
              precision    recall  f1-score   support

         0.0       0.88      0.98      0.93        45
         1.0       0.97      0.85      0.91        40

    accuracy                           0.92        85
   macro avg       0.93      0.91      0.92        85
weighted avg       0.92      0.92      0.92        85



In [84]:
import pickle
# Save the trained model to a file
with open('model.pkl', 'wb') as file:
    pickle.dump(rf_classifier, file)